In [1]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import BingGroundingTool
from dotenv import load_dotenv

In [2]:
# Load environment variables from .env file
load_dotenv()

True

In [3]:
# Create an Azure AI Client from a connection string, copied from your Azure AI Foundry project.
# It should be in the format "<HostName>;<AzureSubscriptionId>;<ResourceGroup>;<HubName>"
# Customers need to login to Azure subscription via Azure CLI and set the environment variables

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["PROJECT_CONNECTION_STRING"],
)

In [ ]:
# [START create_agent_with_bing_grounding_tool]
bing_connection = project_client.connections.get(connection_name=os.environ["BING_CONNECTION_NAME"])
conn_id = bing_connection.id

print(conn_id)

/subscriptions/a2adbedd-e5e9-44a5-ba32-6b2175e3018a/resourceGroups/rg-sc-vgi/providers/Microsoft.MachineLearningServices/workspaces/prj-sc-vgi/connections/binggroundingscvgi


In [ ]:
# Initialize agent bing tool and add the connection id
bing = BingGroundingTool(connection_id=conn_id)

In [ ]:
# Create agent with the bing tool and process assistant run
with project_client:
    agent = project_client.agents.create_agent(
        model=os.environ["O_MODEL_DEPLOYMENT_NAME"],
        name="my-assistant",
        instructions="You are a helpful assistant",
        tools=bing.definitions,
        headers={"x-ms-enable-preview": "true"},
    )
    # [END create_agent_with_bing_grounding_tool]

    print(f"Created agent, ID: {agent.id}")

    # Create thread for communication
    thread = project_client.agents.create_thread()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content="What is the latest Microsoft stock price?",
    )
    print(f"Created message, ID: {message.id}")

    # Create and process agent run in thread with tools
    run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
    print(f"Run finished with status: {run.status}")

    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Delete the assistant when done
    project_client.agents.delete_agent(agent.id)
    print("Deleted agent")

   # Fetch and log all messages
    messages = project_client.agents.list_messages(thread_id=thread.id)
    # Print all messages in the thread
    for msg in reversed(messages.data):
        role = msg.role.capitalize()
        content = msg.content[0]['text']['value'] if msg.content else ''
        print(f"{role}: {content}\n")

Created agent, ID: asst_xbkfT9WVuvs2zWmuHCjeffPg
Created thread, ID: thread_HZaLMUFpuPm4UY4M7Bw4gurY
Created message, ID: msg_nc6SkTLGSUMvgYahtaFISY49
Run finished with status: RunStatus.COMPLETED
Deleted agent
User: What is the latest Microsoft stock price?

Assistant: The latest published price for Microsoft (MSFT) stock is $388.45 as of the most recent trading session【3:1†source】.

